In [ ]:
# ==========================================
# [Step 1] 安装依赖 & 挂载云盘
# ==========================================
import os
import shutil
import time
import re
import json
import requests
import datetime
import subprocess
from concurrent.futures import ThreadPoolExecutor, as_completed
from google.colab import drive
from google.colab import runtime


print("⏳ 正在安装 Whisper (Turbo版) 及 FFmpeg...")
!pip install git+https://github.com/openai/whisper.git -q
!sudo apt update && sudo apt install ffmpeg -q

print("📂 正在挂载 Google Drive...")
drive.mount('/content/drive')
print("✅ 环境准备就绪！请继续运行下一单元格。")

In [ ]:
# ==========================================
# [Step 2] 全局配置 & 函数定义
# ==========================================

# 1. 原始文件名 (填入 Drive 里真实的文件名，包含空格/问号都没关系)
AUDIO_FILENAME = "example.mp3"

# 2. Drive 文件夹路径 (不带末尾斜杠)
DRIVE_FOLDER = "/content/drive/MyDrive/podcast"

# 3. API 设置 (请修改)
API_KEY = "******************"
BASE_URL = "https://**********************/v1"
MODEL_NAME = "gpt-4.1-mini"

# 4. 并发与重试
MAX_WORKERS = 5   # 并发线程数
MAX_RETRIES = 2   # 失败重试轮次

# --- 以下为工具函数，无需修改 ---

def sanitize_filename(name):
    """文件名清洗：只保留字母、数字、下划线、点号"""
    base, ext = os.path.splitext(name)
    safe_base = re.sub(r'[^\w]', '_', base)
    safe_base = re.sub(r'_+', '_', safe_base).strip('_')
    return safe_base + ext

def seconds_to_lrc(seconds):
    """将秒数转为 LRC 格式 [mm:ss.xx]"""
    m = int(seconds // 60)
    s = seconds % 60
    return f"[{m:02d}:{s:05.2f}]"

def format_duration(seconds):
    return str(datetime.timedelta(seconds=int(seconds)))

print(f"⚙️ 配置已加载。目标文件: {AUDIO_FILENAME}")

In [ ]:
# ==========================================
# [Step 3] 自动化 Drive 文件名清洗
# ==========================================

# 计算原始路径
original_path = os.path.join(DRIVE_FOLDER, AUDIO_FILENAME)

# 计算安全文件名
safe_name = sanitize_filename(AUDIO_FILENAME)
safe_path = os.path.join(DRIVE_FOLDER, safe_name)

if not os.path.exists(original_path) and not os.path.exists(safe_path):
    print(f"❌ 错误：在 Drive 路径下找不到文件：\n{original_path}")
    print("请检查文件名和路径是否完全正确。")
else:
    # 逻辑：如果当前名字不安全，且安全名文件不存在，则重命名
    if AUDIO_FILENAME != safe_name:
        if os.path.exists(safe_path):
            print(f"⚠️ 检测到 Drive 上已存在安全名文件，将直接使用：\n   -> {safe_name}")
            AUDIO_FILENAME = safe_name
        else:
            print(f"🔄 正在 Google Drive 上执行重命名 (为了兼容性)...")
            print(f"   🔴 原名: {AUDIO_FILENAME}")
            print(f"   🟢 新名: {safe_name}")
            try:
                os.rename(original_path, safe_path)
                print("✅ Drive 重命名成功！")
                AUDIO_FILENAME = safe_name # 更新全局变量
            except OSError as e:
                print(f"❌ 重命名失败: {e}")

    # 更新后续步骤需要的路径变量
    LOCAL_AUDIO = f"/content/{safe_name}"
    BASE_NAME = os.path.splitext(safe_name)[0]
    LOCAL_JSON = f"/content/{BASE_NAME}.json"
    LOCAL_LRC = f"/content/{BASE_NAME}.lrc"
    DRIVE_LRC = os.path.join(DRIVE_FOLDER, f"{BASE_NAME}.lrc")

    print(f"\n✅ 文件名检查完毕，当前操作对象: {AUDIO_FILENAME}")

In [ ]:
# ==========================================
# [Step 4] 执行 Whisper 转录 (生成 JSON)
# ==========================================

current_audio_path = os.path.join(DRIVE_FOLDER, AUDIO_FILENAME)

if not os.path.exists(current_audio_path):
    print(f"❌ 无法执行：文件不存在 {current_audio_path}")
else:
    print(f"📋 正在将音频复制到 Colab 本地...")
    shutil.copy2(current_audio_path, LOCAL_AUDIO)

    print(f"\n🚀 开始 Whisper 转录 (Turbo 模型)...")

    start_time = time.time()

    # --output_format json: 生成 JSON 数据
    cmd = f'whisper "{LOCAL_AUDIO}" --model turbo --language English --output_format json --output_dir /content/'
    os.system(cmd)

    end_time = time.time()

    if os.path.exists(LOCAL_JSON):
        print(f"\n✅ 转录完成！耗时: {format_duration(end_time - start_time)}")
        print(f"📄 JSON 数据已生成: {LOCAL_JSON}")
    else:
        print("\n❌ 转录失败，未生成 JSON 文件。")

In [ ]:
# ==========================================
# [Step 5] API 翻译 & LRC 生成 & 自动关机
# ==========================================

if not os.path.exists(LOCAL_JSON):
    print("❌ 错误：找不到 JSON 文件，请先运行 Step 4。")
else:
    print(f"🚀 读取 JSON 并开始并发翻译...")
    t_start = time.time()

    # 1. 读取 JSON
    with open(LOCAL_JSON, 'r', encoding='utf-8') as f:
        whisper_data = json.load(f)
    segments = whisper_data['segments']
    total_segs = len(segments)
    print(f"📝 共 {total_segs} 条字幕需要处理。")

    # 2. 翻译函数 (带延时调整)
    def translate_text(text, idx, is_retry=False):
        # 普通模式休眠 0.2s，重试模式休眠 0.5s
        time.sleep(0.2 if not is_retry else 0.5)

        if not text.strip(): return idx, ""

        headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
        payload = {
            "model": MODEL_NAME,
            "messages": [
                {"role": "system", "content": "Translate the following subtitle to Simplified Chinese."},
                {"role": "user", "content": text}
            ],
            "temperature": 0.3
        }

        try:
            resp = requests.post(f"{BASE_URL}/chat/completions", headers=headers, json=payload, timeout=30)
            resp.raise_for_status()
            return idx, resp.json()['choices'][0]['message']['content'].strip()
        except Exception as e:
            if not is_retry: # 非重试时不打印过多报错
                print(f"⚠️ 片段 {idx} 失败: {str(e)[:50]}...")
            return idx, None

    # 3. 并发翻译
    translation_map = {}

    # 第一轮
    print("\n--- 启动第一轮翻译 ---")
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {executor.submit(translate_text, seg['text'], i): i for i, seg in enumerate(segments)}
        finished = 0
        for f in as_completed(futures):
            finished += 1
            if finished % 20 == 0:
                print(f"   进度: {finished}/{total_segs} ({(finished/total_segs)*100:.0f}%)")
            idx, res = f.result()
            if res: translation_map[idx] = res

    # 补漏 (重试)
    failed_indices = [i for i, seg in enumerate(segments) if i not in translation_map and seg['text'].strip()]
    if failed_indices:
        print(f"\n--- 正在重试 {len(failed_indices)} 个失败片段 ---")
        time.sleep(2)
        with ThreadPoolExecutor(max_workers=2) as executor: # 降低并发
            for f in as_completed({executor.submit(translate_text, segments[i]['text'], i, True): i for i in failed_indices}):
                idx, res = f.result()
                if res:
                    translation_map[idx] = res
                    print(f"✅ 补救成功: ID {idx}")

    # 4. 生成 LRC 内容
    print(f"\n📦 正在组装 LRC 双语字幕...")
    lrc_content = []
    lrc_content.append("[ti:Auto Generated]")
    lrc_content.append("[ar:Whisper + GPT]")
    lrc_content.append("")

    success_count = 0
    for i, seg in enumerate(segments):
        timestamp = seconds_to_lrc(seg['start'])
        en_text = seg['text'].strip()
        zh_text = translation_map.get(i, "")

        # 写入格式：[时间戳]原文 \n [时间戳]译文
        if en_text:
            lrc_content.append(f"{timestamp}{en_text}")
        if zh_text:
            lrc_content.append(f"{timestamp}{zh_text}")
            success_count += 1

    # 5. 保存并回传
    with open(LOCAL_LRC, 'w', encoding='utf-8') as f:
        f.write("\n".join(lrc_content))

    shutil.copy2(LOCAL_LRC, DRIVE_LRC)

    t_end = time.time()
    print(f"\n{'='*30}")
    print(f"✅ 任务全部完成！")
    print(f"📊 翻译覆盖率: {success_count}/{total_segs}")
    print(f"💾 LRC 已保存至 Drive: {DRIVE_LRC}")
    print(f"🕒 翻译耗时: {format_duration(t_end - t_start)}")
    print(f"{'='*30}")

    # --- 自动关闭 ---
    print("\n👋 任务结束，5秒后自动断开 Google Colab 连接...")
    time.sleep(5)
    print("🔌 断开运行时...")
    runtime.unassign()